In [3]:
#This code takes a Project Gutenberg book and uses Wikify to add in links to make an enhanced e-Reader.
#NOTE: THIS CODE CAN TAKE AWHILE TO RUN
#Made for DSCI 511 in December 2018

USER_KEY = 'vvtmxtinhohcxaxqgdrleivlgpqxok'

import requests
import re
from bs4 import BeautifulSoup
from IPython.display import IFrame
import os
import urllib.parse, urllib.request, json
from IPython.core.display import HTML
import re

def Gutentag(ID):
    file_path = 'https://www.gutenberg.org/files/' + ID + '/' + ID + '-h/' + ID + '-h.htm'
    data = requests.get(file_path).content
    with open(ID+'.htm', 'wb') as outfile:
        outfile.write(data.text)
        
def wikify(text, lang="en", threshold=0.8):
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", USER_KEY),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("support", "true"), ("includeCosines", "false"), ("ranges", "false")])
    url = "http://www.wikifier.org/annotate-article"
    req = urllib.request.Request(url, data=data.encode("utf8"))#, method="POST")
    with urllib.request.urlopen(req, timeout = 60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    return response

def build_doc(wiki):
    orig_text = ""
    for i in range(len(wiki['spaces'])):
        orig_text = orig_text + wiki['spaces'][i]
        if i <= len(wiki['words'])-1:
            orig_text = orig_text + wiki['words'][i]
        else:
            continue
    return orig_text

        
def get_links(wiki):
    links = []
    for annot in wiki['annotations']:
        for support_dict in annot['support']:
            links.append((support_dict['chFrom'], support_dict['chTo'], annot['url']))
    return links

def build_link(doc, start,end, dest):
    link = doc[start:end+1]
    format = "<a href="+dest+">"+link+"</a>"
    return format

def embed_link(doc, link):
    start = 0
    n_doc = ""
    for l in link:
        n_doc = (n_doc + doc[start:l[0]]+build_link(doc,l[0],l[1],l[2]))  
        start = l[1]+1
    nn_doc = n_doc+doc[start:]
    return HTML(nn_doc)

def embed_links(wiki):
    doc = build_doc(wiki)
    links = get_links(wiki)
    sorted_links = sorted(links, key=lambda x: x[1], reverse=True)
    valid_links = []
    for i in range(len(sorted_links)-1):
        if sorted_links[i+1][1] < sorted_links[i][0]:
            valid_links.append(sorted_links[0])
    valid_links.append(sorted_links[-1])
    link = valid_links[::-1]
    return embed_link(doc, link)        


def get_out_path(path):
    path_list = path.split('/')
    out_dir = path_list[:-1]
    file_name = path_list[-1]
    file = re.split('.htm', file_name)[0]
    out_name = file + '.json'
    out_dir.append(out_name)
    out_path = '/'.join(out_dir)
    return out_path

def wikify_html(path):
    out_path = get_out_path(path)
    with open(path, 'r') as infile:
        html_data = infile.read()
    soup = BeautifulSoup(html_data, 'html.parser')
    p_tags = soup.find_all('p')
    wikified_doc = []
    for p_tag in p_tags:
        wikified_doc.append(wikify(p_tag.text))
    with open(out_path, 'w') as outfile:
        json.dump(wikified_doc, outfile)   

def get_enhanced_html(path):
    json_path = get_out_path(path)
    if os.path.exists(json_path):
        with open(json_path) as f:
            Wikis = json.load(f) 
    else:
        wikify_html(path)
        with open(json_path) as f:
            Wikis = json.load(f)         
    with open(path, "r") as infile: 
        html_file = infile.read()
    newsoup = BeautifulSoup(html_file, 'html.parser')
    p_tags = newsoup.find_all('p')
    p_tags_index_tuples = [(p_tag, i) for i, p_tag in enumerate(p_tags)]    
    annotations_and_num = [(wiki, len(wiki['annotations'])) for i, wiki in enumerate(Wikis)]    
    annotated_paragraph_text = []
    for annot in annotations_and_num:
        if annot[1]==True:
            annotated_paragraph_text.append(embed_links(annot[0]).data)
        else:
            annotated_paragraph_text.append(HTML(build_doc(annot[0])).data)
    enhanced_paragraph = []
    for each_par in annotated_paragraph_text:
        html_par = "<p>"+each_par+"</p>"
        enhanced_paragraph.append(BeautifulSoup(html_par, 'html.parser')) 
    i = 0
    for p in newsoup.find_all('p'):
        n_para = enhanced_paragraph[i]
        if p.string:
            p.string.replace_with(n_para)
        else:
            continue
        i += 1
        if i == len(p_tags)-1:
            break
        else:
            continue
    with open(path[:-4]+"Enhanced.html", 'wb') as f:
        f.write(newsoup.renderContents())
        
    with open(path[:-4]+"Enhanced.html", "r") as f: 
        enhanced_html_file = f.read()
        
def enhanced_book(book_id):
    path = book_id+".htm"
    if os.path.exists(path):
        pass
    else:
        Gutentag(book_id)
    get_enhanced_html(path)
    return  IFrame(path[:-4]+"_Enhanced.html", width=800, height=200)



In [4]:
enhanced_book("19033")